In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [3]:
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [4]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [5]:
# d3 = [273, 372, 221]

In [6]:
ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']

In [7]:
pbar = tqdm()

0it [00:00, ?it/s]

In [8]:
not_ran = [380, 343, 360, 159, 346]

In [9]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in [284]:
    not_ran_operators = []
    # if agency in ran_operators:
    #     print(f'already ran: {agency}')
    #     continue
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        rt_day.filter_formatted = ', PM Peak, Feb 08 (Tue)' ##TODO fix hardcode
        rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == agency)).Name.iloc[0]
        m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])
        m.save(f'./speedmaps/{agency}_02_08_pm_peak.html')
        rt_day.stop_segment_speed_view.to_parquet(f'./speedmap_views/{agency}_02_08_pm_peak.parquet') ## early step towards scaling
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 284...
found parquet
found parquet
found parquet
found parquet
found_parquet
less than 1km of data
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
could not generate delays for trip 257117
Columns must be same length as key
could not generate delays for trip 257127
Columns must be same length as key
could not generate delays for trip 255934
Columns must be same length as key
view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}
speed above 80 for shape 32100, dropping
negative speed for shape 32100, dropping
speed above 80 for shape 32140, dropping
negative speed for shape 32140,

/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:3961: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:3961: RuntimeWarning: invalid value encountered in subtract


speed above 80 for shape 32116, dropping
negative speed for shape 32116, dropping
negative speed for shape 32163, dropping
speed above 80 for shape 32183, dropping
negative speed for shape 32183, dropping


TopologyException: assigned depths do not match at -116060.80090243505 1239.5472101856335
TopologyException: assigned depths do not match at -117953.55795454726 1648.8320321653728
/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
Input geom 1 is INVALID: Self-intersection at or near point -112002.82831454017 10863.90563969472 (-112002.82831454016559 10863.905639694719866)
<A>
MULTIPOLYGON (((-114084.0060600302240346 11813.8892132562868937, -114133.6894329562346684 11835.6277149225752510, -114083.3837823768844828 11855.8846031285356730, -114084.0060600302240346 11813.8892132562868937)), ((-112001.2387243993725860 10858.0429960649416898, -112014.0182606407324784 10905.1758165411702066, -111999.0615781328669982 10919.9168357853995985, -112001.2387243993725860 10858.0429960649416898)), ((-112002.8088594820874278 10863.8859000062984705, -112014.0182606407324784

complete for agency: 284


/tmp/ipykernel_1805/2123817923.py:15: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


In [ ]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')